# Explore the Transformed Play by Play Data

## Setup - Paths - Dependencies

In [82]:
from config import recent_play_by_play, recent_clean_db

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Load Play by Play data
pbp_raw_df = pd.read_csv(recent_play_by_play, low_memory=False)


data_folder = os.path.join('..', 'data') # data folder
temp_folder = os.path.join('..', 'TEMP') # temp folder

# pbp_raw_df.head()


In [83]:
#### HOTFIX TO CLEAN 

### Drop games Exhibition games from dataset
# see two problem teams St Anselm and Assumption - if Game_ID contains either of the teams drop the row
pbp_raw_df = pbp_raw_df[~pbp_raw_df['Game_ID'].str.contains('St Anselm|Assumption')]

# Fix Problem with a few Primary_team values
# Substitute 'UCN' and 'UCN Gustafsson' with 'Connecticut'
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'UCN', 'Primary_team'] = 'Connecticut'
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'UCN Gustafsson', 'Primary_team'] = 'Connecticut'
# Replace the weird 'PRI de la' with 'Princeton'
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'PRI de la', 'Primary_team'] = 'Princeton'


### HOTFIX TO ADD SECONDARY TEAM TO EVERY ROW
df = pbp_raw_df
# Create a new column 'Secondary_team' and initialize it with None
df['Secondary_team'] = None
# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Extract the game ID
    game_id = row['Game_ID']
    # Determine the home and away teams based on the game ID
    away_team = game_id.split('-')[3]
    home_team = game_id.split('-')[4]
    # Assign the opposite team to the 'Secondary_team' column based on the 'Primary_team'
    if row['Primary_team'] == home_team:
        df.at[index, 'Secondary_team'] = away_team
    else:
        df.at[index, 'Secondary_team'] = home_team

pbp_raw_df = df


### HOTFIX TO DEAL WITH St. Cloud State having the period when it shouldn't
# if St. Cloud State is in Primary_team or Secondary_team replace with 'St Cloud State'BaseExceptionGroup
pbp_raw_df.loc[pbp_raw_df['Primary_team'] == 'St. Cloud State', 'Primary_team'] = 'St Cloud State'
pbp_raw_df.loc[pbp_raw_df['Secondary_team'] == 'St. Cloud State', 'Secondary_team'] = 'St Cloud State'



In [84]:
# Rename to df for convenience
df = pbp_raw_df

# Filter for relevant events: Faceoffs and Goals
faceoff_events = df[df['Event_type'] == 'Faceoff']
goal_events = df[df['Event_type'] == 'Goal']

# Merge faceoff events with subsequent goal events within the same game and period
merged_df = pd.merge(
    faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_faceoff', '_goal')
)

# Calculate the time difference between faceoff and goal
merged_df['time_diff'] = merged_df['Time_goal'] - merged_df['Time_faceoff']

# Filter only instances where the goal happens after the faceoff
merged_df = merged_df[merged_df['time_diff'] > 0]


# Count how often a goal is scored within 10 seconds and 5 seconds of a faceoff
goals_within_10s = (merged_df['time_diff'] <= 10).sum()
goals_within_5s = (merged_df['time_diff'] <= 5).sum()

# Count the number of times each team has scored a goal within 5 seconds of a faceoff
teams_scoring_within_5s = merged_df[merged_df['time_diff'] <= 5]['Primary_team_goal'].value_counts()

# Display results
# goals_within_10s, goals_within_5s, teams_scoring_within_5s


# Data Elporation

## Faceoff Danger
- Table of how many times a team has scored within 3,5, 7, and 10 seconds of a faceoff

In [85]:
# Recalculate the time difference properly
merged_df = pd.merge(
    faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_faceoff', '_goal')
)

# Ensure that the goal occurs after the faceoff in time
merged_df = merged_df[merged_df['Time_goal'] > merged_df['Time_faceoff']]

# Calculate the correct time difference between faceoff and goal
merged_df['time_diff'] = merged_df['Time_goal'] - merged_df['Time_faceoff']

# Count goals per team in different time frames
teams_goal_counts = merged_df.groupby('Primary_team_goal')['time_diff'].agg(
    Scored_within_1s=lambda x: (x <= 1).sum(),
    Scored_within_2s=lambda x: (x <= 2).sum(),
    Scored_within_3s=lambda x: (x <= 3).sum(),
    Scored_within_4s=lambda x: (x <= 4).sum(),
    Scored_within_5s=lambda x: (x <= 5).sum(),
    Scored_within_6s=lambda x: (x <= 6).sum(),
    Scored_within_7s=lambda x: (x <= 7).sum(),
    Scored_within_8s=lambda x: (x <= 8).sum(),
    Scored_within_9s=lambda x: (x <= 9).sum(),
    Scored_within_10s=lambda x: (x <= 10).sum()
).reset_index()

# Add the total goals scored by each team
total_goals_per_team = goal_events['Primary_team'].value_counts().reset_index()
total_goals_per_team.columns = ['Primary_team_goal', 'Total_Goals']

# Merge with the team goal counts
teams_goal_counts = teams_goal_counts.merge(total_goals_per_team, on='Primary_team_goal', how='left')



In [86]:
teams_goal_counts.head()

,Primary_team_goal,Scored_within_1s,Scored_within_2s,Scored_within_3s,Scored_within_4s,Scored_within_5s,Scored_within_6s,Scored_within_7s,Scored_within_8s,Scored_within_9s,Scored_within_10s,Total_Goals
0,Air Force,0,0,0,2,2,2,4,5,6,8,55
1,Alaska,0,1,1,3,4,6,6,6,8,9,54
2,Alaska Anchorage,0,0,0,0,0,1,1,2,2,2,55
3,American Intl,0,0,0,0,1,1,1,2,2,4,57
4,Arizona State,0,0,0,0,1,1,1,1,1,2,90


In [87]:
# Calculate the percentage for each goal duration
for sec in range(1, 11):
    col = f'Scored_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_counts[pct_col] = teams_goal_counts[col] / teams_goal_counts['Total_Goals']

# Rearrange columns in an interleaved order: original value then its percentage
cols = ['Primary_team_goal', 'Total_Goals'] + [
    item for sec in range(1, 11)
    for item in (f'Scored_within_{sec}s', f'Scored_within_{sec}s_pct')
]
teams_goal_counts = teams_goal_counts[cols]

# Rename Primary_team_goal to Team
teams_goal_counts.rename(columns={'Primary_team_goal': 'Team'}, inplace=True)

In [88]:
# team_goal_counts.sample(10)

### Save Resulting Table as CSV

In [89]:
### Save to temp folder
teams_goal_counts.to_csv(os.path.join(temp_folder, 'team_goal_counts.csv'), index=False)




## Goals Allowed Just after faceoffs

In [90]:
## Reset df to a clean copy of the full pbp data
df = pbp_raw_df

# Fill the Secondary team column with the opposite team 
# If the Primary team is 'Home', the Secondary team is 'Away' and vice versa based on Game_ID
# Away is after third '-' in game_id. home is after 4th
# check primary team against game id and put opposite in secondary

# # Create a new column 'Secondary_team' and initialize it with None
# df['Secondary_team'] = None
# # Loop through each row in the DataFrame
# for index, row in df.iterrows():
#     # Extract the game ID
#     game_id = row['Game_ID']
#     # Determine the home and away teams based on the game ID
#     away_team = game_id.split('-')[3]
#     home_team = game_id.split('-')[4]
#     # Assign the opposite team to the 'Secondary_team' column based on the 'Primary_team'
#     if row['Primary_team'] == home_team:
#         df.at[index, 'Secondary_team'] = away_team
#     else:
#         df.at[index, 'Secondary_team'] = home_team


# Filter for relevant events: Faceoffs and Goals
faceoff_events = df[df['Event_type'] == 'Faceoff']
goal_events = df[df['Event_type'] == 'Goal']

# Check the first few rows to verify the new column
# df.head()

In [91]:

# Merge faceoff events with subsequent goal events within the same game and period
merged_allowed_df = pd.merge(
    faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_faceoff', '_goal')
)

# Calculate the time difference between faceoff and goal
merged_allowed_df['time_diff'] = merged_allowed_df['Time_goal'] - merged_allowed_df['Time_faceoff']

# Filter only instances where the goal happens after the faceoff
merged_allowed_df = merged_allowed_df[merged_allowed_df['time_diff'] > 0]

merged_allowed_df.head()
# Count goals per team in different time frames
teams_goal_allowed = merged_allowed_df.groupby('Secondary_team_goal')['time_diff'].agg(
    within_1s=lambda x: (x <= 1).sum(),
    within_2s=lambda x: (x <= 2).sum(),
    within_3s=lambda x: (x <= 3).sum(),
    within_4s=lambda x: (x <= 4).sum(),
    within_5s=lambda x: (x <= 5).sum(),
    within_6s=lambda x: (x <= 6).sum(),
    within_7s=lambda x: (x <= 7).sum(),
    within_8s=lambda x: (x <= 8).sum(),
    within_9s=lambda x: (x <= 9).sum(),
    within_10s=lambda x: (x <= 10).sum()
).reset_index()

## LEGACY
# Add the total goals scored by each team 
# total_goals_per_team = goal_events['Secondary_team'].value_counts().reset_index()
# total_goals_per_team.columns = ['Secondary_team', 'Total_Goals']

# Merge with the team goal counts
 

# teams_goal_allowed.head()



In [92]:
# Add the season total of goals allowed by each team
total_goals_allowed_per_team = goal_events['Secondary_team'].value_counts().reset_index()
total_goals_allowed_per_team.columns = ['Secondary_team_goal', 'Total_Goals_Allowed']

# add the data into the teams_goal_allowed dataframe
teams_goal_allowed = teams_goal_allowed.merge(total_goals_allowed_per_team, on='Secondary_team_goal', how='left')


### Merge Two Datatables

In [93]:
# Rename columns for clarity in the allowed goals DataFrame
teams_goal_allowed.columns = ['Team', 'Allowed_within_1s', 'Allowed_within_2s', 'Allowed_within_3s',
                               'Allowed_within_4s', 'Allowed_within_5s', 'Allowed_within_6s', 'Allowed_within_7s',
                               'Allowed_within_8s', 'Allowed_within_9s', 'Allowed_within_10s', 'Total_Goals_Allowed']

# Check the data
teams_goal_allowed.sample(10)


,Team,Allowed_within_1s,Allowed_within_2s,Allowed_within_3s,Allowed_within_4s,Allowed_within_5s,Allowed_within_6s,Allowed_within_7s,Allowed_within_8s,Allowed_within_9s,Allowed_within_10s,Total_Goals_Allowed
51,RIT,0,1,1,3,5,6,7,8,9,10,104
58,Stonehill,0,0,0,1,3,3,3,4,6,9,92
26,Long Island,0,0,0,0,0,0,0,1,2,3,60
45,Ohio State,0,0,1,3,4,4,4,5,6,6,58
61,Western Michigan,0,1,1,1,1,1,1,1,1,1,42
53,Robert Morris,0,0,2,2,2,2,4,4,5,5,79
42,Northeastern,0,1,1,1,4,5,5,6,7,7,68
30,Mercyhurst,0,0,1,3,4,5,8,10,14,18,116
29,Massachusetts,2,2,2,2,2,3,3,4,4,5,69
13,Canisius,0,0,1,4,6,10,10,10,10,11,83


In [94]:
# # Merge the allowed goals DataFrame with the total goals DataFrame
# face_off_goal_success = teams_goal_allowed.merge(teams_goal_counts, on='Team', how='left')
# Calculate the percentage for each goal duration
for sec in range(1, 11):
    col = f'Allowed_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_allowed[pct_col] = teams_goal_allowed[col] / teams_goal_allowed['Total_Goals_Allowed']


# Merge the allowed goals DataFrame with the total goals DataFrame
face_off_goal_success = teams_goal_allowed.merge(teams_goal_counts, on='Team', how='left')




In [95]:
## Save Final Table to Data Folder
# Get today's date as a string
today = pd.Timestamp.now().strftime('%Y-%m-%d')
face_off_goal_success.to_csv(os.path.join(data_folder, f'face_off_goal_success_{today}.csv'), index=False)

# Examine table
# face_off_goal_success.head(10)

# Check info
# face_off_goal_success.info()

### AFTER PP SUCCESS - Within 10 seconds of end of PP

In [96]:
### Get all relevant rows from pbp data
# Filter for relevant events: PP-Start, PP-End and Goals
df = pbp_raw_df

# Filter for relevant events: PP-Start, PP-End and Goals
pp_start_events = df[df['Event_type'] == 'PP - Start']
pp_end_events = df[df['Event_type'] == 'PP - End']
goal_events = df[df['Event_type'] == 'Goal']
# Merge PP-Start events with subsequent goal events within the same game and period
merged_pp_df = pd.merge(
    pp_start_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_pp_start', '_goal')
)

# Calculate the time difference between PP-Start and goal
merged_pp_df['time_diff'] = merged_pp_df['Time_goal'] - merged_pp_df['Time_pp_start']
# Filter only instances where the goal happens after the PP-End
merged_pp_df = merged_pp_df[merged_pp_df['time_diff'] > 0]
# Count goals per team in different time frames
teams_goal_pp = merged_pp_df.groupby('Primary_team_goal')['time_diff'].agg(
    Scored_within_1s=lambda x: (x <= 1).sum(),
    Scored_within_2s=lambda x: (x <= 2).sum(),
    Scored_within_3s=lambda x: (x <= 3).sum(),
    Scored_within_4s=lambda x: (x <= 4).sum(),
    Scored_within_5s=lambda x: (x <= 5).sum(),
    Scored_within_6s=lambda x: (x <= 6).sum(),
    Scored_within_7s=lambda x: (x <= 7).sum(),
    Scored_within_8s=lambda x: (x <= 8).sum(),
    Scored_within_9s=lambda x: (x <= 9).sum(),
    Scored_within_10s=lambda x: (x <= 10).sum(),
    Scored_within_15s=lambda x: (x <= 15).sum(),
    Scored_within_20s=lambda x: (x <= 20).sum(),
    Scored_within_30s=lambda x: (x <= 30).sum()
).reset_index()

# Add the total goals scored by each team
total_goals_per_team = goal_events['Primary_team'].value_counts().reset_index()
total_goals_per_team.columns = ['Primary_team_goal', 'Total_Goals']
# Merge with the team goal counts
teams_goal_pp = teams_goal_pp.merge(total_goals_per_team, on='Primary_team_goal', how='left')

# Calculate the percentage for each goal duration
# Define the seconds list to include 1-10, 15, 20, and 30 seconds
secs = list(range(1, 11)) + [15, 20, 30]

# Calculate the percentage for each goal duration
for sec in secs:
    col = f'Scored_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_pp[pct_col] = teams_goal_pp[col] / teams_goal_pp['Total_Goals']

# Rearrange columns in an interleaved order: original value then its percentage
cols = ['Primary_team_goal', 'Total_Goals'] + [
    item for sec in secs
    for item in (f'Scored_within_{sec}s', f'Scored_within_{sec}s_pct')
]
teams_goal_pp = teams_goal_pp[cols]

# Rename Primary_team_goal to Team
teams_goal_pp.rename(columns={'Primary_team_goal': 'Team'}, inplace=True)
# teams_goal_pp.head(10)
teams_goal_pp.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Team                   64 non-null     object 
 1   Total_Goals            64 non-null     int64  
 2   Scored_within_1s       64 non-null     int64  
 3   Scored_within_1s_pct   64 non-null     float64
 4   Scored_within_2s       64 non-null     int64  
 5   Scored_within_2s_pct   64 non-null     float64
 6   Scored_within_3s       64 non-null     int64  
 7   Scored_within_3s_pct   64 non-null     float64
 8   Scored_within_4s       64 non-null     int64  
 9   Scored_within_4s_pct   64 non-null     float64
 10  Scored_within_5s       64 non-null     int64  
 11  Scored_within_5s_pct   64 non-null     float64
 12  Scored_within_6s       64 non-null     int64  
 13  Scored_within_6s_pct   64 non-null     float64
 14  Scored_within_7s       64 non-null     int64  
 15  Scored_w

In [97]:
### Count how many Power Play goals each team has scored by looking at the 'Goal_Conditions' column of pbp_raw_df 
# # if it contains POWER-PLAY it is a PP Goal


# Count how many Power Play goals each team has scored by looking at the 'Goal_Conditions' column of pbp_raw_df
# # if it contains POWER-PLAY it is a PP Goal
# Initialize a DataFrame to store the counts
pp_goal_counts = pd.DataFrame(columns=['Team', 'PP_Goals'])
# Loop through each team
for team in df['Primary_team'].unique():
    # Count the number of goals for the team that contain 'POWER-PLAY' in 'Goal_Conditions'
    pp_goals = df[(df['Primary_team'] == team) & (df['Goal_Conditions'].str.contains('POWER-PLAY', na=False))]
    pp_goal_counts = pd.concat([pp_goal_counts, pd.DataFrame({'Team': [team], 'PP_Goals': [len(pp_goals)]})])

# Reset the index
pp_goal_counts.reset_index(drop=True, inplace=True)
# Merge the PP goal counts with the main DataFrame
teams_goal_pp = teams_goal_pp.merge(pp_goal_counts, on='Team', how='left')
# Check the data
# teams_goal_pp.sample(10)




In [98]:
# pbp_raw_df.head()

In [99]:
# goal_events.head()

In [100]:
### Do the same for allowed goals
# Merge PP-End events with subsequent goal events within the same game and period
merged_pp_allowed_df = pd.merge(
    pp_end_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team', 'Secondary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_pp_end', '_goal')
)

# Calculate the time difference between PP-End and goal
merged_pp_allowed_df['time_diff'] = merged_pp_allowed_df['Time_goal'] - merged_pp_allowed_df['Time_pp_end']
# Filter only instances where the goal happens after the PP-End
merged_pp_allowed_df = merged_pp_allowed_df[merged_pp_allowed_df['time_diff'] > 0]
# Count goals per team in different time frames
teams_goal_pp_allowed = merged_pp_allowed_df.groupby('Primary_team_goal')['time_diff'].agg(
    Allowed_within_1s=lambda x: (x <= 1).sum(),
    Allowed_within_2s=lambda x: (x <= 2).sum(),
    Allowed_within_3s=lambda x: (x <= 3).sum(),
    Allowed_within_4s=lambda x: (x <= 4).sum(),
    Allowed_within_5s=lambda x: (x <= 5).sum(),
    Allowed_within_6s=lambda x: (x <= 6).sum(),
    Allowed_within_7s=lambda x: (x <= 7).sum(),
    Allowed_within_8s=lambda x: (x <= 8).sum(),
    Allowed_within_9s=lambda x: (x <= 9).sum(),
    Allowed_within_10s=lambda x: (x <= 10).sum(),
    Allowed_within_15s=lambda x: (x <= 15).sum(),
    Allowed_within_20s=lambda x: (x <= 20).sum(),
    Allowed_within_30s=lambda x: (x <= 30).sum()
).reset_index()

# Rename Primary_team_goal to Team
teams_goal_pp_allowed.rename(columns={'Primary_team_goal': 'Team'}, inplace=True)
# Add the total goals allowed by each team
total_goals_allowed_per_team = goal_events['Secondary_team'].value_counts().reset_index()
total_goals_allowed_per_team.columns = ['Secondary_team', 'Total_Goals_Allowed']
# Rename to simply Team
total_goals_allowed_per_team.rename(columns={'Secondary_team': 'Team'}, inplace=True)

total_goals_allowed_per_team
# Merge with the team goal counts
teams_goal_pp_allowed = teams_goal_pp_allowed.merge(total_goals_allowed_per_team, on='Team', how='left')

teams_goal_pp_allowed.head()
# teams_goal_pp_allowed.info()

# # look at mean min and max
# teams_goal_pp_allowed.describe()




,Team,Allowed_within_1s,Allowed_within_2s,Allowed_within_3s,Allowed_within_4s,Allowed_within_5s,Allowed_within_6s,Allowed_within_7s,Allowed_within_8s,Allowed_within_9s,Allowed_within_10s,Allowed_within_15s,Allowed_within_20s,Allowed_within_30s,Total_Goals_Allowed
0,Air Force,0,0,0,0,0,0,0,0,0,0,1,1,1,76.0
1,Alaska,0,0,0,0,0,0,0,0,0,0,0,0,0,62.0
2,Alaska Anchorage,0,0,0,0,0,0,0,0,0,0,1,1,2,85.0
3,American Intl,0,0,0,1,1,1,1,1,1,1,4,5,6,79.0
4,Arizona State,0,0,1,1,1,1,1,1,1,1,2,3,4,56.0


In [101]:
## Calculate the percentage for each goal duration
# Define the seconds list to include 1-10, 15, 20, and 30 seconds
secs = list(range(1, 11)) + [15, 20, 30]

# Calculate the percentage for each goal duration
for sec in secs:
    col = f'Allowed_within_{sec}s'
    pct_col = f'{col}_pct'
    teams_goal_pp_allowed[pct_col] = teams_goal_pp_allowed[col] / teams_goal_pp_allowed['Total_Goals_Allowed']

# Rearrange columns in an interleaved order: original value then its percentage
cols = ['Team', 'Total_Goals_Allowed'] + [
    item for sec in secs
    for item in (f'Allowed_within_{sec}s', f'Allowed_within_{sec}s_pct')
]
teams_goal_pp_allowed = teams_goal_pp_allowed[cols]
# Merge the allowed goals DataFrame with the total goals DataFrame
face_off_allowed_goal_pp = teams_goal_pp_allowed.merge(teams_goal_pp, on='Team', how='left')
face_off_allowed_goal_pp.head(10)

,Team,Total_Goals_Allowed,Allowed_within_1s,Allowed_within_1s_pct,Allowed_within_2s,Allowed_within_2s_pct,Allowed_within_3s,Allowed_within_3s_pct,Allowed_within_4s,Allowed_within_4s_pct,...,Scored_within_9s_pct,Scored_within_10s,Scored_within_10s_pct,Scored_within_15s,Scored_within_15s_pct,Scored_within_20s,Scored_within_20s_pct,Scored_within_30s,Scored_within_30s_pct,PP_Goals
0,Air Force,76.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,1,0.018182,1,0.018182,2,0.036364,12
1,Alaska,62.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.018519,1,0.018519,1,0.018519,2,0.037037,2,0.037037,11
2,Alaska Anchorage,85.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.018182,1,0.018182,2,0.036364,5,0.090909,8,0.145455,14
3,American Intl,79.0,0,0.000000,0,0.000000,0,0.000000,1,0.012658,...,0.000000,1,0.017544,1,0.017544,2,0.035088,4,0.070175,14
4,Arizona State,56.0,0,0.000000,0,0.000000,1,0.017857,1,0.017857,...,0.011111,1,0.011111,1,0.011111,1,0.011111,6,0.066667,23
5,Army,76.0,1,0.013158,1,0.013158,2,0.026316,2,0.026316,...,0.000000,1,0.013699,3,0.041096,3,0.041096,4,0.054795,12
6,Augustana,42.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,2,0.027778,3,0.041667,5,0.069444,12
7,Bemidji State,73.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.048387,3,0.048387,4,0.064516,5,0.080645,5,0.080645,14
8,Bentley,59.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.014706,1,0.014706,4,0.058824,4,0.058824,5,0.073529,16
9,Boston College,41.0,0,0.000000,1,0.024390,1,0.024390,1,0.024390,...,0.012500,1,0.012500,1,0.012500,1,0.012500,3,0.037500,12


In [102]:
## Count the number of PP goals allowed by each team and add to the DataFrame
# Initialize a DataFrame to store the counts
pp_goal_allowed_counts = pd.DataFrame(columns=['Team', 'PP_Goals_Allowed'])
# Loop through each team
for team in df['Secondary_team'].unique():
    # Count the number of goals for the team that contain 'POWER-PLAY' in 'Goal_Conditions'
    pp_goals_allowed = df[(df['Secondary_team'] == team) & (df['Goal_Conditions'].str.contains('POWER-PLAY', na=False))]
    pp_goal_allowed_counts = pd.concat([pp_goal_allowed_counts, pd.DataFrame({'Team': [team], 'PP_Goals_Allowed': [len(pp_goals_allowed)]})])
# Reset the index
pp_goal_allowed_counts.reset_index(drop=True, inplace=True)

# Merge the PP goal counts with the main DataFrame
teams_goal_pp_allowed = teams_goal_pp_allowed.merge(pp_goal_allowed_counts, on='Team', how='left')
# Check the data
teams_goal_pp_allowed.sample(10)

,Team,Total_Goals_Allowed,Allowed_within_1s,Allowed_within_1s_pct,Allowed_within_2s,Allowed_within_2s_pct,Allowed_within_3s,Allowed_within_3s_pct,Allowed_within_4s,Allowed_within_4s_pct,...,Allowed_within_9s_pct,Allowed_within_10s,Allowed_within_10s_pct,Allowed_within_15s,Allowed_within_15s_pct,Allowed_within_20s,Allowed_within_20s_pct,Allowed_within_30s,Allowed_within_30s_pct,PP_Goals_Allowed
53,Robert Morris,79.0,0,0.000000,0,0.000000,1,0.012658,1,0.012658,...,0.012658,1,0.012658,1,0.012658,2,0.025316,3,0.037975,18
57,St. Thomas,72.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,0,0.000000,0,0.000000,1,0.013889,17
48,Princeton,54.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,0,0.000000,2,0.037037,2,0.037037,10
32,Miami,93.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,0,0.000000,1,0.010753,2,0.021505,22
58,Stonehill,92.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.010870,1,0.010870,1,0.010870,1,0.010870,2,0.021739,19
16,Colorado College,67.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,1,0.014925,4,0.059701,5,0.074627,12
50,Quinnipiac,57.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,0,0.000000,0,0.000000,2,0.035088,8
15,Colgate,74.0,0,0.000000,0,0.000000,1,0.013514,2,0.027027,...,0.054054,4,0.054054,5,0.067568,7,0.094595,8,0.108108,13
12,Brown,48.0,1,0.020833,1,0.020833,1,0.020833,1,0.020833,...,0.041667,2,0.041667,3,0.062500,3,0.062500,4,0.083333,16
61,Western Michigan,42.0,0,0.000000,0,0.000000,0,0.000000,0,0.000000,...,0.023810,2,0.047619,2,0.047619,2,0.047619,2,0.047619,2


In [103]:
## Merge the Scored and allowed datasets into a single final table
# Merge the scored and allowed DataFrames
post_power_play_success = teams_goal_pp_allowed.merge(teams_goal_pp, on='Team', how='left')
# Check the data
post_power_play_success.sample(10)

,Team,Total_Goals_Allowed,Allowed_within_1s,Allowed_within_1s_pct,Allowed_within_2s,Allowed_within_2s_pct,Allowed_within_3s,Allowed_within_3s_pct,Allowed_within_4s,Allowed_within_4s_pct,...,Scored_within_9s_pct,Scored_within_10s,Scored_within_10s_pct,Scored_within_15s,Scored_within_15s_pct,Scored_within_20s,Scored_within_20s_pct,Scored_within_30s,Scored_within_30s_pct,PP_Goals
32,Miami,93.0,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0.022727,1,0.022727,1,0.022727,3,0.068182,5,0.113636,14
3,American Intl,79.0,0,0.0,0,0.000000,0,0.000000,1,0.012658,...,0.000000,1,0.017544,1,0.017544,2,0.035088,4,0.070175,14
31,Merrimack,71.0,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,2,0.033333,2,0.033333,4,0.066667,14
1,Alaska,62.0,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0.018519,1,0.018519,1,0.018519,2,0.037037,2,0.037037,11
2,Alaska Anchorage,85.0,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0.018182,1,0.018182,2,0.036364,5,0.090909,8,0.145455,14
11,Bowling Green,65.0,0,0.0,1,0.015385,1,0.015385,1,0.015385,...,0.000000,0,0.000000,1,0.015873,5,0.079365,6,0.095238,18
55,St Cloud State,101.0,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0.018519,1,0.018519,3,0.055556,3,0.055556,5,0.092593,11
13,Canisius,83.0,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0.014286,1,0.014286,1,0.014286,1,0.014286,3,0.042857,20
35,Michigan Tech,54.0,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0.000000,0,0.000000,0,0.000000,1,0.014925,3,0.044776,19
23,Holy Cross,60.0,0,0.0,0,0.000000,1,0.016667,1,0.016667,...,0.025641,2,0.025641,3,0.038462,5,0.064103,8,0.102564,19


In [105]:
post_power_play_success.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Team                    64 non-null     object 
 1   Total_Goals_Allowed     63 non-null     float64
 2   Allowed_within_1s       64 non-null     int64  
 3   Allowed_within_1s_pct   63 non-null     float64
 4   Allowed_within_2s       64 non-null     int64  
 5   Allowed_within_2s_pct   63 non-null     float64
 6   Allowed_within_3s       64 non-null     int64  
 7   Allowed_within_3s_pct   63 non-null     float64
 8   Allowed_within_4s       64 non-null     int64  
 9   Allowed_within_4s_pct   63 non-null     float64
 10  Allowed_within_5s       64 non-null     int64  
 11  Allowed_within_5s_pct   63 non-null     float64
 12  Allowed_within_6s       64 non-null     int64  
 13  Allowed_within_6s_pct   63 non-null     float64
 14  Allowed_within_7s       64 non-null     int6

In [104]:
total_goals_allowed_per_team.head(50)
total_goals_allowed_per_team.tail(20)

,Team,Total_Goals_Allowed
44,Bentley,59
45,Ohio State,58
46,Quinnipiac,57
47,Dartmouth,57
48,Providence,57
49,Michigan State,57
50,Clarkson,56
51,Arizona State,56
52,Michigan Tech,54
53,Omaha,54
